# Pan05 Temp test

In [1]:
import intake

import matplotlib.pyplot as plt
import xarray as xr
import cf_xarray as cfxr
import numpy as np

import cmocean as cm
import cartopy.crs as ccrs

import cartopy.feature as cft
import matplotlib.path as mpath

In [2]:
#Importing data
pancontrol005 = intake.open_esm_datastore('/home/272/kc5856/catalog_dir/panant_005_zstar_access_yr2.json', columns_with_iterables=['variable'])
pantrial005 = intake.open_esm_datastore('/home/272/kc5856/panantrial/panant_005_zstar_MWonly.json', columns_with_iterables=['variable'])

In [3]:
#Starting a dask client
from os import environ
environ["PYTHONWARNINGS"] = "ignore"

from dask.distributed import Client
client = Client(threads_per_worker=1)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39215 instead
  warnings.warn(


In [4]:
#Time variant variables #Can this be condensed?
control005_temp = pancontrol005.search(variable=["thetao"],frequency="1mon",file_id=["XXXXXXXX_ocean_month_z"]).to_dask()
trial005_temp = pantrial005.search(variable="thetao", frequency="1mon").to_dask()

In [6]:
#Time invariant variables
data005_fixed = pancontrol005.search(variable=["areacello","deptho"],path=".*output021.*").to_dask()
vol005 = pancontrol005.search(variable=["volcello"],path=".*output021.*",file_id=["XXXXXXXX_ocean_month_z"]).to_dask()

In [7]:
#Area weighting
areacello005 = data005_fixed.areacello
thick005 = vol005 / areacello005
thick005 = thick005.load()

Shelf mask

In [9]:
def shelf_mask_isobath(var, model_dict):

    paths = {
             "mom5": "/g/data/ik11/grids/Antarctic_slope_contour_1000m.npz",
             "mom6_01": "/g/data/ik11/grids/Antarctic_slope_contour_1000m_MOM6_01deg.nc",
            "mom6_005": "/g/data/ik11/grids/Antarctic_slope_contour_1000m_MOM6_005deg.nc"
             }

    var = var.cf.sel({'latitude': slice(-90, -59)})

    if paths[model_dict][-3:] == '.nc':
        shelf_mask = xr.open_dataset(paths[model_dict])['contour_masked_above']
    else:
        contour_file = np.load(paths[model_dict])
        shelf_mask = xr.DataArray(contour_file['contour_masked_above'],
                                  coords = var.coords, 
                                  dims = var.dims,
                                  name = 'contour_masked_above')
    
    shelf_mask = xr.where(shelf_mask == 0, 1, 0)
    masked_var = var * shelf_mask
    
    return masked_var, shelf_mask

In [10]:
depth005 = data005_fixed.deptho
land_mask = (depth005*0).fillna(1)
depth_shelf, shelf_mask = shelf_mask_isobath(depth005, 'mom6_005')

Loading top 500m average

In [11]:
# Slicing first
cont005_500_time = control005_temp.thetao.sel(z_l=slice(0,520)).sel(time=slice('2001-01-01','2010-12-31')).sel(yh=slice(None,-59))
pert005_500_time = trial005_temp.thetao.sel(z_l=slice(0,520)).sel(time=slice('2001-01-01','2010-12-31')).sel(yh=slice(None,-59))

In [12]:
# Weighting and means
cont005_500_time = cont005_500_time.weighted(thick005['z_l'].fillna(0)).mean(dim='z_l').where(shelf_mask == 1).weighted(areacello005.fillna(0)).mean(dim=('xh','yh'))
pert005_500_time = pert005_500_time.weighted(thick005['z_l'].fillna(0)).mean(dim='z_l').where(shelf_mask == 1).weighted(areacello005.fillna(0)).mean(dim=('xh','yh'))

In [ ]:
cont005_500_time = cont005_500_time.load()
pert005_500_time = pert005_500_time.load()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/client.py:3357: UserWarning: Sending large graph of size 105.95 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
cont005_500_time.plot(label="Control")
pert005_500_time.plot(label="Perturbation")
plt.xlim(380,4000)
plt.legend(loc="lower left")
plt.title("Panan005 Time Series of Temperature for shelf top 500m")